<p><img style="padding: 0 15px; float: left;" src="Images/FTLogo300.jpg" alt="FT Crusader Logo" Width='140' Height= '250'/></p> 
<p> <h2> Frogtown DataLoad Workbook, 05/06/18</h2> <a name="tc"></a>
<p> <h3> By Frogtown Crusader (Abu Nayeem)</h3>

This is just clean coded process going over the uploading steps assuming all the preliminary steps after decoder are made


## Table of contents <a name="content"></a>
* [Data Setup](#setup)
* [Create Variables](#create)
* [Intersection Table: Preparation](#intersection)
* [Address Table: Preparation](#address)
* [Preliminary Data Cleaning](#pre)


### Data Setup <a name="setup"></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import HTML
from IPython.display import display
import requests # library to handle requests

#Load Data
#df_crime = pd.read_csv('Datasets/Crime_Incident_Report_-_Dataset.csv')

#rename columns
#cols= ['Case','Date','Time','Code','IncType','Incident','Grid','NNum','Neighborhood','Block','CallDispCode','CallDisposition', 'Count']
#df_crime.columns= cols

#selection for Frogtown and nearby area
#df=df_crime.query('Grid in [66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110]')

#Socarata upload Method
from sodapy import Socrata

#New Upload Method Get Information from Socrata API
client = Socrata("information.stpaul.gov", None)

#Easier to bulk upload
results = client.get("gppb-g9cg", limit=1000000)
df = pd.DataFrame.from_records(results)
# Find Max Date Value
results = client.get("gppb-g9cg", limit=1)
r_max = pd.DataFrame.from_records(results)

#rename columns [Note the order of Columns have changed]
cols= ['Block','CallDispCode','CallDisposition','Case','Code', 'Count','Date','Grid','Incident','IncType','Neighborhood','NNum','Time']
df.columns= cols
df = df.astype({"Case": int, "Code": int, "Grid":float, "NNum":int,"Count":int})
#select respective Grids of interest
df=df.query("Grid in (66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110)")

df.head(4)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,Neighborhood,NNum,Time
2,29X PHALEN BD,A,Advised,19180716,9954,1,2019-08-13T23:43:48.000,92.0,Proactive Police Visit,Proactive Police Visit,5 - Payne/Phalen,5,2019-08-13T23:43:48.000
7,47X MARION ST,G,Gone on Arrival,19180704,600,1,2019-08-13T23:20:24.000,110.0,Theft,"Theft, Except Auto Theft",8 - Summit/University,8,2019-08-13T23:20:24.000
9,43X UNIVERSITY AV E,A,Advised,19180697,9954,1,2019-08-13T23:16:31.000,92.0,Proactive Police Visit,Proactive Police Visit,5 - Payne/Phalen,5,2019-08-13T23:16:31.000
12,58X DALE ST N,A,Advised,19180675,9954,1,2019-08-13T22:45:38.000,88.0,Proactive Police Visit,Proactive Police Visit,7 - Thomas/Dale(Frogtown),7,2019-08-13T22:45:38.000


### Create New Variables <a name="create"></a>

In [2]:
#Add Time Variables
df= df[df.Case != 18254093] #messed up time variable

#Convert Date to Datetime!
from datetime import datetime

df['DateTime']= pd.to_datetime(df['Date']) # Create new column called DateTime
df['Year']= df['DateTime'].dt.year #create year column
df['DayofWeek']=df['DateTime'].dt.dayofweek #create day of the week column where default 0=Monday
df['Weekend'] = df['DayofWeek'].apply(lambda x: 1 if (x>4)  else 0) #Create a weekend category
df['Month'] = df['DateTime'].dt.month # Create Month Category
df['Day'] = df['DateTime'].dt.day #Create Day of the Current month
df['DayYear'] = df['DateTime'].dt.dayofyear  #Create Day of the year (0-365)
df['Day_Max'] = df.iloc[0,-1] #selects uptodate day; NOTE: the data is sorted chronologically

#Hour Data
df['TimeHour']= pd.to_datetime(df['Time'])
df['Hour'] = df['TimeHour'].dt.hour.astype(int) #Create Hour Colum
df['LateNight'] = df['Hour'].apply(lambda x: 1 if (x>21 or x<5)  else 0) #Latenight designation from 10Pm to 6PM

#Creating the intersection Column. Note: the Block column has the address information
df.Block = df.Block.astype(str) #first change the type to string
df['Block']= df['Block'].str.lower() #lowercase string to create uniformity

#While scanning the data I noticed that all intersections had "&" 
df['Intersection'] = df['Block'].apply(lambda x: 1 if '&' in x else 0) #intersection

df.head(5)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,DayofWeek,Weekend,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection
2,29x phalen bd,A,Advised,19180716,9954,1,2019-08-13T23:43:48.000,92.0,Proactive Police Visit,Proactive Police Visit,...,1,0,8,13,225,225,2019-08-13 23:43:48,23,1,0
7,47x marion st,G,Gone on Arrival,19180704,600,1,2019-08-13T23:20:24.000,110.0,Theft,"Theft, Except Auto Theft",...,1,0,8,13,225,225,2019-08-13 23:20:24,23,1,0
9,43x university av e,A,Advised,19180697,9954,1,2019-08-13T23:16:31.000,92.0,Proactive Police Visit,Proactive Police Visit,...,1,0,8,13,225,225,2019-08-13 23:16:31,23,1,0
12,58x dale st n,A,Advised,19180675,9954,1,2019-08-13T22:45:38.000,88.0,Proactive Police Visit,Proactive Police Visit,...,1,0,8,13,225,225,2019-08-13 22:45:38,22,1,0
14,50x rice st,A,Advised,19180662,9954,1,2019-08-13T22:32:26.000,90.0,Proactive Police Visit,Proactive Police Visit,...,1,0,8,13,225,225,2019-08-13 22:32:26,22,1,0


### Prepare Intersection table <a name="intersection"></a>

In [3]:
#Load clean intersections key
df_Intersection_key = pd.read_csv('Datasets/Intersection_key_clean.csv')

# Create a new dateframe specifying only intersections from primary dataset
dfI=df.query('Intersection ==1')
print('The intersection table dimension are ' + str(dfI.shape))
#print(dfI.Block.head(10))


#Split the strings
new=dfI['Block'].str.split("& ", n = 1, expand = True) 
dfI['Inter2']= new[1]
new=dfI['Block'].str.split(" ", n = 1, expand = True) #Note the code specifies the first time a space occured
dfI['Inter1']=new[0]

#Create the IndexKey; recall we prepared the IntersectionKey where it considers any order
dfI['IndexKey']= dfI['Inter1']+ '_' + dfI['Inter2']
dfI.reset_index()
dfI=pd.merge(dfI, df_Intersection_key, on='IndexKey', how='left')



The intersection table dimension are (5919, 25)


In [4]:
#Drop rows with missing coordinates
dfI=dfI[dfI['Coordinates'].notnull()]

# Separate Latitude and Longitude 
new=dfI['Coordinates'].str.split(",", n = 1, expand = True) 
# making seperate first name column from new data frame 
dfI['Latitude']= pd.to_numeric(new[0]) #pd.to_numeric convert it to float
dfI['Longitude']= pd.to_numeric(new[1])

#Renaming columns
dfI['Block']=dfI['OutputKey'] #for practical purposes it makes sense
Drop_col=['Inter2','Inter1', 'IndexKey', 'Coordinates', 'OutputKey']
dfI_Final=dfI.drop(Drop_col, axis=1,)
dfI_Final.head(5)


,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection,Latitude,Longitude
2,linden_mtairy,A,Advised,19179845,9954,1,2019-08-12T23:34:19.000,92.0,Proactive Police Visit,Proactive Police Visit,...,8,12,224,225,2019-08-12 23:34:19,23,1,1,44.959025,-93.095111
3,milton_thomas,A,Advised,19179838,9954,1,2019-08-12T23:24:59.000,87.0,Proactive Police Visit,Proactive Police Visit,...,8,12,224,225,2019-08-12 23:24:59,23,1,1,44.959361,-93.139031
4,marshall_hamline,A,Advised,19179791,9954,1,2019-08-12T22:16:13.000,106.0,Proactive Police Visit,Proactive Police Visit,...,8,12,224,225,2019-08-12 22:16:13,22,1,1,44.948530,-93.156810
5,western_university,A,Advised,19179718,9954,1,2019-08-12T20:13:27.000,90.0,Proactive Police Visit,Proactive Police Visit,...,8,12,224,225,2019-08-12 20:13:27,20,0,1,44.955870,-93.116148
6,victoria_thomas,A,Advised,19179604,9954,1,2019-08-12T17:20:30.000,87.0,Proactive Police Visit,Proactive Police Visit,...,8,12,224,225,2019-08-12 17:20:30,17,0,1,44.959355,-93.136492


### Prepare Address Decoder Table <a name="address"></a>

It is so remarkably short

In [5]:
#Split Data
dfW=df.query('Intersection==0')

#Load Complete Decoder Key
df_C= pd.read_csv('Datasets/SemiKey.csv')
df_C= df_C[['Block','Latitude','Longitude']]

# Merge with the dataset and remove missing values
dC=pd.merge(dfW, df_C, on='Block', how='left')
dC=dC.fillna('Mi')
dC=dC.query('Latitude != "Mi"')

#Bringing the data back together 
fg= dfI.append(dC, ignore_index=True)

### Preliminary Pre-Cleaning steps <a name="clean"></a>

It is good to do the data pre-cleaning steps here to reduce clutter on a working notebook. A few edits include renaming some values, clustering crimes together and creating some dummies. It is saved in a csv, which is used for execution

In [6]:
#Few Quick Edits
fg.CallDisposition.loc[(fg['CallDisposition'] == 'G - Gone on Arrival')] = 'Gone on Arrival'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'A - Advised')] = 'Advised'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'RR - Report Written')] = 'Report Written'
fg.Incident.loc[(fg['Incident'] == 'Simple Asasult Dom.')] = 'Simple Assault Dom.'
fg.Incident.loc[(fg['Incident'] == 'Graffiti')] = 'Vandalism'
fg.Incident.loc[fg["Incident"].isin([ "Rape","Agg. Assault",'Homicide'])]= 'Violent'
fg.Incident.loc[fg["Incident"].isin(["Simple Assault Dom.","Agg. Assault Dom."])]= 'Domestic Assault'

#[fg["Incident"].isin(["Simple Assault Dom.", "Rape"])


#Create a dummy for each crime category
fg= pd.concat([fg,pd.get_dummies(fg['Incident'])], axis=1)
fg= pd.concat([fg,pd.get_dummies(fg['CallDisposition'])], axis=1)

fg.to_csv('Datasets/FGCrime_Final.csv', encoding='utf-8', index=False)

In [7]:
fg.head(4)

,Block,CallDispCode,CallDisposition,Case,Code,Coordinates,Count,Date,DateTime,Day,...,Domestic Assault,Narcotics,Proactive Police Visit,Robbery,Theft,Vandalism,Violent,Advised,Gone on Arrival,Report Written
0,linden_mtairy,A,Advised,19179845,9954,"44.959025, -93.095111",1,2019-08-12T23:34:19.000,2019-08-12 23:34:19,12,...,0,0,1,0,0,0,0,1,0,0
1,milton_thomas,A,Advised,19179838,9954,"44.959361, -93.139031",1,2019-08-12T23:24:59.000,2019-08-12 23:24:59,12,...,0,0,1,0,0,0,0,1,0,0
2,marshall_hamline,A,Advised,19179791,9954,"44.948530, -93.156810",1,2019-08-12T22:16:13.000,2019-08-12 22:16:13,12,...,0,0,1,0,0,0,0,1,0,0
3,western_university,A,Advised,19179718,9954,"44.955870, -93.116148",1,2019-08-12T20:13:27.000,2019-08-12 20:13:27,12,...,0,0,1,0,0,0,0,1,0,0


In [ ]:
Ward1.query('ElectionDescription in ("11/08/2016 - STATE GENERAL", "08/09/2016 - STATE PRIMARY","11/07/2017 - MUNICIPAL GENERAL","11/07/2017 - SCHOOL DISTRICT GENERAL"  ,"11/08/2016 - STATE GENERAL","08/09/2016 - STATE PRIMARY","11/03/2015 - MUNICIPAL GENERAL","11/03/2015 - SCHOOL DISTRICT GENERAL","11/08/2011 - MUNICIPAL GENERAL","11/06/2018 - STATE GENERAL","08/14/2018 - STATE PRIMARY")' 